# Generacion del dataset para el Dashboard

In [160]:
#Importamos las librerias necesarias.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import ydata_profiling
from ydata_profiling import ProfileReport
import langdetect
from langdetect import detect
import googletrans
from googletrans import Translator
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('all')


%matplotlib inline
%config IPCompleter.greedy=True

Importamos los datasets EDA

In [161]:
#Importamos los datasets EDA
coursera = pd.read_csv(r"C:\Users\miche\OneDrive\Escritorio\Henry\Proyecto_DatAnalitycs\Datasets_EDA\coursera_EDA.csv")
edx = pd.read_csv(r"C:\Users\miche\OneDrive\Escritorio\Henry\Proyecto_DatAnalitycs\Datasets_EDA\edx_EDA.csv")
udemy = pd.read_csv(r"C:\Users\miche\OneDrive\Escritorio\Henry\Proyecto_DatAnalitycs\Datasets_EDA\udemy_EDA.csv")

Verificamos que fueron importamos de forma correcta

In [162]:
coursera.head(2)

,name,course_id,n_reviews,mean_rating,language,course_difficulty,course_students_enrolled,MOOC
0,Machine Learning,machine-learning,12677.0,4.739923,English,Mixed,3200000,Coursera
1,Indigenous Canada,indigenous-canada,1460.0,4.816438,Indonesia,Mixed,51000,Coursera


In [163]:
edx.head(2)

,title,summary,n_enrolled,course_type,institution,instructors,level,subject,language,subtitles,course_effort,course_length,price,certificate,certificate_price,course_description,course_syllabus
0,How to Learn Online,Learn essential strategies for successful onli...,124980.0,Self-paced on your time,edX,Nina Huntemann-Robyn Belair-Ben Piscopo,Introductory,Education & Teacher Training,English,English,2–3 hours per week,2 Weeks,FREE,YES,49.0,"Designed for those who are new to elearning, t...",Welcome - We start with opportunities to meet ...
1,Programming for Everybody (Getting Started wit...,"This course is a ""no prerequisite"" introductio...",293864.0,Self-paced on your time,The University of Michigan,Charles Severance,Introductory,Computer Science,English,English,2–4 hours per week,7 Weeks,FREE,YES,49.0,This course aims to teach everyone the basics ...,NaN


In [164]:
udemy.head(2)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,published_year,published_month_name,published_day_name,published_day,language,rating
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18 20:58:58,Business Finance,2017,Enero,Miercoles,18,English,0.76
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09 16:34:20,Business Finance,2017,Marzo,Jueves,9,English,0.96


* COURSERA

In [165]:
coursera.head(2)

,name,course_id,n_reviews,mean_rating,language,course_difficulty,course_students_enrolled,MOOC
0,Machine Learning,machine-learning,12677.0,4.739923,English,Mixed,3200000,Coursera
1,Indigenous Canada,indigenous-canada,1460.0,4.816438,Indonesia,Mixed,51000,Coursera


Borramos la informacion que no es relevante para realizar la segmentacion por MOOC

Borramos las filas que no aportan valor a nuestro EDA

In [166]:
coursera.drop(labels=["course_id", "n_reviews", "course_students_enrolled"], axis=1, inplace=True)

Cambiamos formatos y palabras en el dataset para que quede acorde a los otros MOOC a analizar

In [167]:
#Cambios
coursera.rename({"name": "nombre_curso", "mean_rating": "rating_curso", "language":"idioma_curso", "course_difficulty":"nivel_curso"}, axis= 1,inplace=True)
coursera.insert(1, "precio_curso", value=0)
coursera["nivel_curso"] = coursera["nivel_curso"].str.replace("Mixed", "All Levels")

In [168]:
coursera.head(2)

,nombre_curso,precio_curso,rating_curso,idioma_curso,nivel_curso,MOOC
0,Machine Learning,0,4.739923,English,All Levels,Coursera
1,Indigenous Canada,0,4.816438,Indonesia,All Levels,Coursera


* Edx

In [169]:
edx.head(2)

,title,summary,n_enrolled,course_type,institution,instructors,level,subject,language,subtitles,course_effort,course_length,price,certificate,certificate_price,course_description,course_syllabus
0,How to Learn Online,Learn essential strategies for successful onli...,124980.0,Self-paced on your time,edX,Nina Huntemann-Robyn Belair-Ben Piscopo,Introductory,Education & Teacher Training,English,English,2–3 hours per week,2 Weeks,FREE,YES,49.0,"Designed for those who are new to elearning, t...",Welcome - We start with opportunities to meet ...
1,Programming for Everybody (Getting Started wit...,"This course is a ""no prerequisite"" introductio...",293864.0,Self-paced on your time,The University of Michigan,Charles Severance,Introductory,Computer Science,English,English,2–4 hours per week,7 Weeks,FREE,YES,49.0,This course aims to teach everyone the basics ...,NaN


Borramos la informacion que no es relevante para realizar la segmentacion por MOOC

Borramos las filas que no aportan valor a nuestro EDA

In [170]:
edx.drop(labels=["summary", "n_enrolled", "course_type", "institution", "instructors", "subject", "subtitles", "course_effort", "course_length", "price", "certificate", "course_description","course_syllabus"], axis=1, inplace=True)

In [171]:
edx.head(2)

,title,level,language,certificate_price
0,How to Learn Online,Introductory,English,49.0
1,Programming for Everybody (Getting Started wit...,Introductory,English,49.0


Cambiamos formatos y palabras en el dataset para que quede acorde a los otros MOOC a analizar

In [172]:
edx.rename({"title":"nombre_curso", "level":"nivel_curso", "language":"idioma_curso", "certificate_price":"precio_curso"}, axis=1, inplace=True)
edx.nivel_curso = edx.nivel_curso.str.replace("Introductory", "Beginner")
edx.insert(2,"rating_curso", value=0)
edx.insert(3, "MOOC", value="EDX")

In [173]:
edx = edx.reindex(columns=["nombre_curso", "precio_curso", "rating_curso", "idioma_curso","nivel_curso", "MOOC"])

In [174]:
edx.head(2)

,nombre_curso,precio_curso,rating_curso,idioma_curso,nivel_curso,MOOC
0,How to Learn Online,49.0,0,English,Beginner,EDX
1,Programming for Everybody (Getting Started wit...,49.0,0,English,Beginner,EDX


* Udemy

In [175]:
udemy.head(2)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,published_year,published_month_name,published_day_name,published_day,language,rating
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18 20:58:58,Business Finance,2017,Enero,Miercoles,18,English,0.76
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09 16:34:20,Business Finance,2017,Marzo,Jueves,9,English,0.96


Borramos la informacion que no es relevante para realizar la segmentacion por MOOC

Borramos las filas que no aportan valor a nuestro EDA

In [176]:
udemy.drop(labels=["course_id", "url", "is_paid", "num_subscribers", "num_reviews", "num_lectures", "content_duration", "published_timestamp", "subject", "published_year", "published_month_name","published_day_name" ,"published_day"], axis=1, inplace=True)

In [177]:
udemy.rename({"course_title":"nombre_curso", "level":"nivel_curso", "language":"idioma_curso", "price":"precio_curso", "rating":"rating_curso"}, axis=1, inplace=True)
udemy["nivel_curso"] = udemy["nivel_curso"].str.replace("Beginner Level","Beginner")
udemy["nivel_curso"] = udemy["nivel_curso"].str.replace("Intermediate Level","Intermediate")
udemy["nivel_curso"] = udemy["nivel_curso"].str.replace("Expert Level", "Advanced")
udemy.insert(5, "MOOC", value="Udemy")
udemy = udemy.reindex(columns=["nombre_curso", "precio_curso", "rating_curso", "idioma_curso","nivel_curso", "MOOC"])

In [178]:
udemy

,nombre_curso,precio_curso,rating_curso,idioma_curso,nivel_curso,MOOC
0,Ultimate Investment Banking Course,200,0.76,English,All Levels,Udemy
1,Complete GST Course & Certification - Grow You...,75,0.96,English,All Levels,Udemy
2,Financial Modeling for Business Analysts and C...,45,0.14,English,Intermediate,Udemy
3,Beginner to Pro - Financial Analysis in Excel ...,95,0.87,English,All Levels,Udemy
4,How To Maximize Your Profits Trading Options,200,0.98,English,Intermediate,Udemy
...,...,...,...,...,...,...
3671,Learn jQuery from Scratch - Master of JavaScri...,100,0.21,English,All Levels,Udemy
3672,How To Design A WordPress Website With No Codi...,25,0.15,English,Beginner,Udemy
3673,Learn and Build using Polymer,40,0.78,English,All Levels,Udemy
3674,CSS Animations: Create Amazing Effects on Your...,50,0.98,English,All Levels,Udemy


Creamos un nuevo Dataframe unico con todos

In [181]:
df_final = pd.concat([coursera, edx, udemy], axis=0)

Verificamos y exportamos.

In [185]:
df_final.to_csv("eda.csv", index=False)

Controlador

In [190]:
controlador = df_final["MOOC"]

In [191]:
controlador.to_csv("controlar.csv", index=False)